In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

In [2]:
country     = pd.read_csv("country.csv")
league      = pd.read_csv("league.csv")
match       = pd.read_csv("match.csv")
match['date']  = pd.to_datetime(match['date'])
ptemp       = pd.read_csv("player_attributes.csv") 
ptemp['date']  = pd.to_datetime(ptemp['date']) # add ending date/time for which player info is valid
player_att  = ptemp.assign(next_date=ptemp.groupby('player_id').date.shift(1))
player      = pd.read_csv("player.csv")
ttemp       = pd.read_csv("team_attributes.csv")  
ttemp['date']  = pd.to_datetime(ttemp['date'])  # add ending date/time for which team info is valid
team_att    = ttemp.assign(next_date=ttemp.groupby('team_id').date.shift(-1))
team        = pd.read_csv("team.csv")

In [3]:
def get_att(attribute, date, name):
    # Go to player_att, find attribute entry on row with name and contains date
    name_check = player_att['player_id'] == name
    history = player_att[['date', attribute, 'next_date']][name_check]
    date_check = (history['date'] < date) & ((history['next_date'] != history['next_date']) | (history['next_date'] > date))
    history = history[date_check]
    if history.empty: return np.nan
    return history[attribute].iloc[0]

def avg_func(row, attribute):
    date = row['date']
    vals = [get_att(attribute, date, row[player]) for player in row.index[1:12] if row[player] == row[player]]
    vals = [x for x in vals if x == x]
    # prevent division by 0 for empty lists
    if (len(vals) == 0): return np.nan
    return sum(vals)/len(vals)

def calc_player_info(team, attribute, start_time, end_time):
    # create home and away dataframes containing matches within stated time interval
    rel_home = match[(match['home_team_id'] == team)]
    rel_home = rel_home[(rel_home['date'] != rel_home['date']) | ((rel_home['date'] > start_time) & (rel_home['date'] < end_time))]
    rel_home = pd.concat((rel_home.loc[:, 'date'], rel_home.loc[:, 'home_player_1':'home_player_11']), axis=1)
    rel_away = match[(match['away_team_id'] == team)]
    rel_away = rel_away[(rel_away['date'] != rel_away['date']) | ((rel_away['date'] > start_time) & (rel_away['date'] < end_time))]
    rel_away = pd.concat((rel_away.loc[:, 'date'], rel_away.loc[:, 'away_player_1':'away_player_11']), axis=1)

    # pull a df of players in set
    rel_home['out'] = ''
    rel_away['out'] = ''
    rel_home['out'] = rel_home.apply(lambda x: avg_func(x, attribute), axis=1)
    rel_away['out'] = rel_away.apply(lambda x: avg_func(x, attribute), axis=1)
    print("one run")
    return (rel_home['out'].mean() + rel_away['out'].mean())/2

In [4]:
num_attributes = [att for att in player_att.select_dtypes('float64')]

for att in num_attributes:
    team_att[att] = team_att.apply(lambda x: calc_player_info(x['team_id'], att, x['date'], x['next_date']), axis=1)
    print("Completed attribute: ", att)
    team_att.to_csv("team_attributes_floats")

one run
one run
one run
one run
one run
one run
one run
one run
one run
one run
one run
one run
one run
one run
one run
one run


KeyboardInterrupt: 